In [1]:

import pandas as pd
from pandasql import sqldf
import pandasql as sql
import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Layout, GridspecLayout
#from itables import init_notebook_mode, show
from IPython.display import display, HTML, Markdown
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import json
import plotly.express as px
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import panel as pn
pn.extension() 
pd.options.display.max_rows
pd.set_option('display.max_colwidth', 1)

# Bigfoot and Campsite data
campsite_df = pd.read_csv('CAMPSITE_INFO.csv')
bf_enc_df = pd.read_csv('BF_ENCOUNTER_INFO.csv')

# UI Widgets
# Output widget
output = widgets.Output()

# State widget dropdown setup
state_widget=widgets.Dropdown(
        options=[ 'AK', 'AL', 'AR',	 'AZ',	 'CA',	 'CO',	 'CT',	 'FL',	 'GA',	 'KY',	 'MA',	 'MD',	 'OK',	 'TN',	 'VT'],
        #value= 'State',
        description='Choose State: ',
        disabled=False 
)

state_widget_layout = widgets.Layout(
    width='50%',
    margin='0 0 10px 100'
)

state_widget.layout = state_widget_layout



# Campsite Checkbox setup
campsites = ['Lodging', 'RVs', 'Tents']

campsite_checkboxes = [widgets.Checkbox(value=False, description=campsite) for campsite in campsites]

campsites_label = widgets.Label(value="Campsite Type:")
campsites_layout = widgets.VBox(campsite_checkboxes)

campsite_box=widgets.VBox([campsites_label, campsites_layout])



#Features Checkbox setup
features = ['Forest', 'Mountains', 'Lake', 'River or Creek']

feature_checkboxes = [widgets.Checkbox(value=False, description=feature) for feature in features]

features_label = widgets.Label(value="Natural Features:")
features_layout = widgets.VBox(feature_checkboxes)

feature_box=widgets.VBox([features_label, features_layout])


# Activity Checkbox setup
activities = ['Boating', 'Fishing', 'Swimming', 'Climbing', 'Biking', 'Hiking']

activity_checkboxes = [widgets.Checkbox(value=False, description=activity) for activity in activities]

activities_label = widgets.Label(value="Activities:")
activities_layout = widgets.VBox(activity_checkboxes)

activities_box=widgets.VBox([activities_label, activities_layout])


# Search Button setup
search_button = widgets.Button(
    value=False,
    description='Find Bigfoot',
    disabled=False,
    layout=Layout(height='auto', width='auto'),
    button_style='success', 
    tooltip='Description',
     
)
search_button.style.button_color = '#03396C'
search_button.style.font_family = 'Arial, sans-serif'
search_button.style.font_size = '22px'

# "Campsite Recommendations" Label
crec_label_html = HTML(
    "<h1 style='background-color: #03396C; color: white; text-align: center; font-family: Arial, sans-serif; line-height: 2; font-size: 24px;'>Campsite Recommendations</h1>"
)

# "Squatch Finder" Label
squatch_label = widgets.Button(
    description="Bigfoot Finder",
    layout=widgets.Layout(height='auto', width='auto'),
    button_style='success', 
)
squatch_label.style.button_color = '#03396C'
squatch_label.style.font_family = 'Arial, sans-serif'
squatch_label.style.font_size = '36px'


# Creates Area Encounter Button for displaying comments from bigfoot sightings in the area    
def create_enc_button(tc_variable):
    comments_button = widgets.Button(
    value=False,
    description='Click here to read about Bigfoot encounters in this area!',
    disabled=False,
    layout=Layout(height='70px', width='1000px'),
    button_style='success', 
    tooltip='Description'  )

    comments_button.style.button_color = '#03396C'
    comments_button.style.font_family = 'Arial, sans-serif'
    comments_button.style.font_size = '22px'

    #display(comments_button)
    
    comments_button.on_click(lambda b: on_comments_click(b, tc_variable, output))
    #comments_button.on_click(get_comments(tc_variable))
    return comments_button

# Retrieves comments when Arean Encounter button is clicked
def on_comments_click(b, tc, output):
    with output:
        get_comments(tc)

# Retrieves comments from .csv file based on the county of the top recommendation    
def get_comments(county_s):
    county=county_s
    query = "SELECT ENC_COMMENTS FROM bf_enc_df WHERE ENC_COUNTY = '"+county+"'"
    comments = sql.sqldf(query)
    display(comments.style.set_properties(**{'text-align': 'left'}))

def error_message (message):
    with output:
        clear_output(wait=True)  
        print(f"Error: {message}")


# Renders Chloropleth Map for showing the nummber of reported encounters by county
def get_map(state):

    if state == 'AL':
        county_map = json.load(open("Counties/GEOJSON/alabama-with-county-boundaries_1083.geojson", 'r'))
        map_df = pd.read_csv('Counties/COUNTY STATS/AL_ENC_STATS.csv')
    elif state == 'AK':
        county_map = json.load(open("Counties/GEOJSON/alaska-with-county-boundaries_1082.geojson", 'r'))
        map_df = pd.read_csv('Counties/COUNTY STATS/AK_ENC_STATS.csv')
    elif state == 'AR':
        county_map = json.load(open("Counties/GEOJSON/arkansas-with-county-boundaries_1084.geojson", 'r')) 
        map_df = pd.read_csv('Counties/COUNTY STATS/AR_ENC_STATS.csv')
    elif state == 'AZ':
        county_map = json.load(open("Counties/GEOJSON/arizona-with-county-boundaries_1085.geojson", 'r')) 
        map_df = pd.read_csv('Counties/COUNTY STATS/AZ_ENC_STATS.csv')
    elif state == 'CA':
        county_map = json.load(open("Counties/GEOJSON/california-with-county-boundaries_1086.geojson", 'r')) 
        map_df = pd.read_csv('Counties/COUNTY STATS/CA_ENC_STATS.csv')
    elif state == 'CO':
        county_map = json.load(open("Counties/GEOJSON/colorado-with-county-boundaries_1087.geojson", 'r')) 
        map_df = pd.read_csv('Counties/COUNTY STATS/CO_ENC_STATS.csv')
    elif state == 'CT':
        county_map = json.load(open("Counties/GEOJSON/connecticut-with-county-boundaries_1088.geojson", 'r')) 
        map_df = pd.read_csv('Counties/COUNTY STATS/CT_ENC_STATS.csv')
    elif state == 'FL':
        county_map = json.load(open("Counties/GEOJSON/florida-with-county-boundaries_1091.geojson", 'r')) 
        map_df = pd.read_csv('Counties/COUNTY STATS/FL_ENC_STATS.csv')
    elif state == 'GA':
        county_map = json.load(open("Counties/GEOJSON/georgia-with-county-boundaries_1092.geojson", 'r')) 
        map_df = pd.read_csv('Counties/COUNTY STATS/GA_ENC_STATS.csv')
    elif state == 'KY':
        county_map = json.load(open("Counties/GEOJSON/kentucky-with-county-boundaries_1100.geojson", 'r')) 
        map_df = pd.read_csv('Counties/COUNTY STATS/KY_ENC_STATS.csv')
    elif state == 'MA':
        county_map = json.load(open("Counties/GEOJSON/massachusetts-with-county-boundaries_1102.geojson", 'r')) 
        map_df = pd.read_csv('Counties/COUNTY STATS/MA_ENC_STATS.csv')
    elif state == 'MD':
        county_map = json.load(open("Counties/GEOJSON/maryland-with-county-boundaries_1103.geojson", 'r')) 
        map_df = pd.read_csv('Counties/COUNTY STATS/MD_ENC_STATS.csv')
    elif state == 'OK':
        county_map = json.load(open("Counties/GEOJSON/oklahoma-with-county-boundaries_1119.geojson", 'r')) 
        map_df = pd.read_csv('Counties/COUNTY STATS/OK_ENC_STATS.csv')
    elif state == 'TN':
        county_map = json.load(open("Counties/GEOJSON/tennessee-with-county-boundaries_1125.geojson", 'r')) 
        map_df = pd.read_csv('Counties/COUNTY STATS/TN_ENC_STATS.csv')
    elif state == 'VT':
        county_map = json.load(open("Counties/GEOJSON/vermont-with-county-boundaries_1130.geojson", 'r')) 
        map_df = pd.read_csv('Counties/COUNTY STATS/VT_ENC_STATS.csv')
    
    county_map['features'][0] 
    
    county_id_map = {}
    for feature in county_map['features']:
        feature['id']= feature['properties']['id']
        county_id_map[feature['properties']['name']] = feature['id']

    map_df['id'] = map_df['ENC_COUNTY'].apply(lambda x: county_id_map[x])
    map_df.head()

    map_df['ENC_DENSITY'] = np.log10(map_df['NUM_EST'])

    map = px.choropleth(map_df, locations='id', geojson=county_map, color='ENC_DENSITY', color_continuous_scale="Viridis", scope='usa', labels={'NUM_ENC':'Number of Encounters','ENC_DENSITY':'Encounter Density'}, hover_name='ENC_COUNTY', hover_data=['NUM_ENC'], title= "Number of Bigfoot Encounters by County")
    map.update_geos(fitbounds='locations', visible=True)
    map.show()
    

#Method for creating string value based on user input
@output.capture(clear_output=True)
def search_click(b):
    
    with output:
        
       
        state = state_widget.value
        if any(campsite.value for campsite in campsite_checkboxes):
            camp_type = f"{' '.join([campsite.description for campsite in campsite_checkboxes if campsite.value])}"
        else:
            print("Please select a campsite type.")
            
        if any(feature.value for feature in feature_checkboxes):
                feat_type = f" ".join([feature.description for feature in feature_checkboxes if feature.value])
        else:
            print("Please select features.")   
            
        if any(activity.value for activity in activity_checkboxes):
            
                act_type = f" ".join([activity.description for activity in activity_checkboxes if activity.value])
        else:
            print("Please select activities.") 
            
        user_input = f" {"'" + state + ' ' + camp_type + ' ' +feat_type +  ' ' + act_type + "'"}"
                
        cosine_algorithm(user_input, output) 
             
    
#Runs cosine algortim for selection of campsites with closest cosine similarity to user input
#Renders campsite recommendations and visuals on the interface
def cosine_algorithm(user_input, output):
    
    #creates new column in campsite df to combine all attributes for each record into a single string
    campsite_df['AttributesCombined'] = (campsite_df['state'].fillna('') + ' ' + campsite_df['Activities'].fillna('')
        + ' ' + campsite_df['camp_type'].fillna('') + ' ' + campsite_df['natural_features'].fillna(''))

    #cosine algorithm
    dv = TfidfVectorizer(stop_words='english')
    dm = dv.fit_transform(campsite_df['AttributesCombined'].values.astype('U'))
    similarities_output = linear_kernel(dv.transform([user_input]), dm).flatten()

    #adds a column for the cosine values associated with each record
    campsite_df['CosineRecs'] = similarities_output
    
    #sorts by campsite with the closest similarity score
    recommended_campsites =campsite_df.sort_values(by='CosineRecs', ascending=False)
    Top5_inState = recommended_campsites.head(50)

    #Sorts top campsites according to the number of bigfoot sightings
    campsites_sorted = Top5_inState.sort_values(by='num_enc', ascending=False)
    top5_campsites=campsites_sorted.head(5)

    #Variables needed for rendering visuals
    top_county = top5_campsites['county'].head(1).iloc[0]
    top_state = top5_campsites['state'].head(1).iloc[0]
 
    #displays Campsite Recommendations to the UI
    display(crec_label_html)
    display(top5_campsites[['id', 'address', 'city', 'county', 'state', 'camp_type', 'natural_features', 'Activities']].style.set_properties(**{'text-align': 'left'}))
    
    #Bigfoot sighting by county map
    get_map(top_state)

    #Bigfoot sighting by season bar chart
    query = "SELECT * FROM bf_enc_df WHERE CONFIRMED_SIGHTING = 'yes'"
    sighting_df = sqldf(query)
    bfdf_count_season = sighting_df.groupby(by=["STATE_ABRV", "ENC_SEASON"]).size().reset_index(name="count")
    season_fig = px.histogram(bfdf_count_season, x="STATE_ABRV", y="count", color="ENC_SEASON", barmode='group', title="Bigfoot Sightings by Season ", labels={'STATE_ABRV' : 'State Abbreviation', 'ENC_SEASON' : 'Season' })
    season_fig.show()
    
    #Cosine Similarity Scatter Plot 
    cosine_fig = px.scatter(campsite_df, x="CosineRecs", y="state", color="state", title="Campsites by Cosine Similarity to User Input", labels={'CosineRecs' : 'Cosine Similarity'})          
    cosine_fig.show()   

    #Area Encounters Button
    comments = create_enc_button(top_county)
    with output:
        display(comments)


# Widget box for formatting UI
user_select_box = widgets.VBox([state_widget, campsite_box, feature_box, activities_box], layout=widgets.Layout(padding='20px'))
results_box = widgets.HBox([output])

#Grid layout for orgainzing UI
grid = GridspecLayout(6, 4, height='900px')
grid[0,:4] = squatch_label
grid[1:5, 3] = user_select_box
grid[1:6, :3] = results_box
grid[5, 3] = search_button

#Displays UI
display(grid)

#Search button action
search_button.on_click(search_click) 




GridspecLayout(children=(Button(button_style='success', description='Bigfoot Finder', layout=Layout(grid_area=…